In [20]:
from qiskit import Aer, execute
from qiskit import QuantumCircuit
from qiskit.quantum_info import operators
from qiskit.quantum_info.analysis import average_data

from scipy.optimize import minimize, rosen
import numpy as np

In [17]:
class VQE:
    
    #回路の作成
    def __init__(self, n_qubits, observable, layer, backend):
        self.n_qubits = n_qubits
        self.OBS = observable
        self.Layer = layer
        self.backend = backend

    def make_circuit(self, Params):

        def make_U_circuit(circ, params):
            for n in range(self.n_qubits):
                param = params[3*n: 3*(n+1)]
                circ.rz(param[0], n)
                circ.rx(param[1], n)
                circ.rz(param[2], n)
            return circ
        
        def make_Ent(circ):
            for n in range(self.n_qubits-1):
                circ.cx(n, n+1)
            return circ
        
        circ = QuantumCircuit(self.n_qubits, self.n_qubits)

        for l in range(self.Layer):
            #parameter circuitの作成
            params = Params[3*self.n_qubits*l : 3*self.n_qubits*(l+1)]
            #entanglementの作成
            make_U_circuit(circ, params)

            if l != self.Layer-1:
                make_Ent(circ)

        circ.measure(circ.qregs[0], circ.cregs[0])

        return circ
    
    ##VQE実行フェーズ
    def outputlayer(self, params):
        circ = self.make_circuit(params)
        counts = execute(circ, backend=self.backend, shots=8192).result().get_counts()
        return average_data(counts, self.OBS)
    
    def initial_params(self):
        init = [0.1 for _ in range(3 * self.Layer * self.n_qubits)]
        return np.array(init)
    
    def minimize(self):
        initial_params = self.initial_params()
        #最適化の実行
        opt_params, opt_cost = classica_minimize(self.outputlayer, initial_params, options={'maxiter': 500})
        circ = self.make_circuit(opt_params)
        counts = execute(circ, backend=self.backend, shots=8192).result().get_counts()
        ans = sorted(counts.items(), key=lambda x: x[1], reverse=True)
        print(ans)
        return opt_cost

#VQE実行フェーズ

In [11]:
def sample_hamiltonian():
    I_mat = np.array([[1, 0], [0, 1]])
    X_mat = np.array([[0, 1], [1, 0]])
    Z_mat = np.array([[1, 0], [0, -1]])
    obs = np.zeros((4,4))
    obs += -3.8505 * np.kron(I_mat, I_mat)
    obs += -0.2288 * np.kron(I_mat, X_mat)
    obs += -1.0466 * np.kron(I_mat, Z_mat)
    obs += 0.2613 * np.kron(X_mat, X_mat)
    obs += 0.2288 * np.kron(X_mat, Z_mat)
    obs += -1.0466 * np.kron(Z_mat, I_mat)
    obs += 0.2288 * np.kron(X_mat, X_mat)
    obs += 0.2356 * np.kron(X_mat, X_mat)
    return obs / 2

In [25]:
def classica_minimize(cost_func, initial_params, options):
    print('classical minimize is starting now...')
    result = minimize(cost_func, initial_params, options=options, method='powell',  tol=1e-6)
    print('opt_cost: {}'.format(result.run))
    print('opt_params: {}'.format(result.x))
    return result['x'], result['fun']

In [26]:
backend = Aer.get_backend('qasm_simulator')
obs = sample_hamiltonian()
vqe = VQE(2, obs, 2, backend)
vqe.minimize()

classical minimize is starting now...


AttributeError: run